In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import os


In [2]:
tf.__version__

'2.0.0'

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_tf = pd.read_csv(TRAIN_DATA_URL)
test_data = pd.read_csv(TEST_DATA_URL)

In [4]:
print(train_tf.shape)
print(test_data.shape)

(627, 10)
(264, 10)


In [5]:
train_data = train_tf[:500]
valid_data = train_tf[500:]

In [6]:
print(train_data.shape)
print(valid_data.shape)

(500, 10)
(127, 10)


In [7]:
y_train= train_data.pop('survived')
y_test = test_data.pop('survived')
y_valid= valid_data.pop('survived')



In [12]:
categorical_columns = ['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
numeric_columns = ['age', 'fare']

feature_columns = []#收集所有feature，以运用到dataset上
#对离散特征进行onehot 编码处理
for column in categorical_columns:
    vocab = train_data[column].unique()#用unique取出唯一值
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(column, vocab)))
    
#categorical_column_with_vocabulary_list提取特征list
# indicator_column 对离散特征进行one_hot编码

#对连续型特征直接封装
for column in numeric_columns:
    feature_columns.append(tf.feature_column.numeric_column(column, dtype = tf.float32))

In [13]:
# 构建数据集.from_tensor_slices()传入字典
def make_datasets(x, y, epochs = 10, shuffle = True, batch_size = 32 ):
    dataset = tf.data.Dataset.from_tensor_slices((dict(x), y))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [14]:
output_dir ='data/baseline_model'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
baseline_estimator = tf.estimator.BaselineClassifier(
        model_dir = output_dir,
        n_classes =2,
        optimizer='Ftrl',)
baseline_estimator.train(input_fn = lambda : make_datasets(
        train_data, y_train, epochs=100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'data/baseline_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x6404f4650>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done c

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at /Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]

Original stack trace for 'IteratorGetNext':
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/ipy

In [ ]:
baseline_estimator.evaluate(input_fn = lambda : make_datasets(
        test_data, y_test, epochs=20))

In [15]:
linear_output_dir ='data/linear_model'
if not os.path.exists(linear_output_dir):
    os.mkdir(linear_output_dir)
linear_estimator = tf.estimator.LinearClassifier(
        model_dir = output_dir,
        n_classes =2,
        optimizer='Ftrl',
        feature_columns=feature_columns)
linear_estimator.train(input_fn = lambda : make_datasets(
        train_data, y_train, epochs=100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'data/baseline_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x640bd69d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to ru

In [ ]:
linear_estimator.evaluate(input_fn = lambda : make_datasets(
        test_data, y_test, epochs=20))

In [ ]:
dnn_output_dir ='data/dnn_model'
if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)
dnn_estimator = tf.estimator.DNNClassifier(
        model_dir = dnn_output_dir,
        n_classes =2,
        activation_fn='relu',
        hidden_units=2,
        optimizer='Ftrl',
        feature_columns=feature_columns)
dnn_estimator.train(input_fn = lambda : make_datasets(
        train_data, y_train, epochs=100))

In [ ]:
dnn_estimator.evaluate(input_fn = lambda : make_datasets(
        test_data, y_test, epochs=20))